# Installation

In [1]:
#!pip install folium
#!pip install altair
#!pip install branca

# Imports

In [480]:
import pandas as pd
import folium
from folium.features import FeatureGroup, VegaLite
from folium import plugins
import branca
import altair as alt
import matplotlib.pyplot as plt
import io

# Example with Circle Markers and Popups with Bar Charts
# Example with Pie Charts as Markers

In [481]:
#### Read 'xlsx' file as pandas dataframe
df = pd.read_excel(r'data/maltratos.xlsx', converters={'Long':str, 'Lat':str}, index_col=None)
df_tr = pd.read_excel(r'data/maltratos_process.xlsx', index_col=None)

# Remove rows with value '0' on column 'Long'
df = df.drop(df['Long'].loc[df['Long']=="0"].index)
# Create a list with the index of df
count_hospitals = list(df.index)

# Create a map
m = folium.Map(location=[22.037635528256533, -79.36879425670136], zoom_start=7, 
               tiles=None)

# Add a title
loc = 'Hospitales con testimonios de violencia obstétrica en Cuba'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)  
m.get_root().html.add_child(folium.Element(title_html))

# Add layer and set dark as default mode
folium.TileLayer(tiles="CartoDB dark_matter", name="Dark", attr= '''
               &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
               contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
               ''').add_to(m)

# Add layer for light mode
folium.TileLayer(tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', 
                 name="Light", attr= '''
                 &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
                 contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
                ''', overlay=False).add_to(m)

# Add fullscreen option
plugins.Fullscreen(
    position="topright",
    title="Pantalla completa",
    title_cancel="Salir de pantalla completa",
    force_separate_button=True,
).add_to(m)

# Add 'Maltratos' group
maltratos = FeatureGroup(name="Maltratos")
maltratos.add_to(m)

# Add 'Consentimiento' group
consentimiento = FeatureGroup(name="Consentimiento", show=False)
consentimiento.add_to(m)

# Add the option to switch layers
folium.LayerControl('topright', collapsed= False).add_to(m)

# List with columns names
columns_list = list(df_tr)

# Empty list for bar charts
charts = []
# Create a chart for each hospital column
for i in range(len(columns_list)):
    charts.append("chart"+'i')
    charts[i] = alt.Chart(df_tr).mark_bar(color='blue', size=20).encode(
            alt.X(columns_list[i+2], title='Hospital', 
                  scale=alt.Scale(domain=[0, 30])),
            alt.Y(columns_list[0], title='Tipos', 
                  sort=alt.EncodingSortField(field=columns_list[i+2], op="count", order='ascending')),
            tooltip=columns_list[1]).interactive().properties(title='Tipos de maltrato por Hospital', 
                                                              width=350, height=300)
    # Save chart as json
    charts[i].to_json()
    if i == len(count_hospitals)-1:
        break

# Add circle markers for each row in df
k = 0
for j, r in df.iterrows():
    folium.CircleMarker([r['Long'], r['Lat']],
    radius=r['Total']/3,
    color='#9b00ff',
    fill_color='#9b00ff',
    fill_opacity=(0.5),
    tooltip=r['Hospital'],
    # Add popup with chart
    popup=folium.Popup(max_width=450, max_heigth=400).add_child(folium.VegaLite(charts[k], width=450, height=300))
    ).add_to(maltratos)
    k += 1

# Read 'xlsx' file as pandas dataframe
df2 = pd.read_excel(r'data/consent.xlsx', converters={'Long':str, 'Lat':str}, index_col=None)
df_tr2 = pd.read_excel(r'data/consent_process.xlsx', index_col=None)

legend_html = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<body>
 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; left: 20px; bottom: 20px;'>
     
<div class='legend-title'>Consentimiento</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#e6194b;opacity:0.9;'></span>No, para ninguno</li>
    <li><span style='background:#00d29f;opacity:0.9;'></span>Sí, para algunos</li>
    <li><span style='background:#ff9300;opacity:0.9;'></span>Sí, para todos</li>

  </ul>
</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: blue;
    }
  .maplegend:hover {
     color:black;
     transform: scale(1.02);
    }
</style>
{% endmacro %}"""

# Add legend to map
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
m.get_root().add_child(legend)

# Convert values from df to arrays
data = []
for i, c in df_tr2.iteritems():
    values = df_tr2[i].to_numpy(dtype=int, na_value=0)
    data.append(values)

# Empty list for pie charts
plots = []
# Create a chart for each hospital column
fig = plt.figure(figsize=(0.5, 0.5))
fig.patch.set_alpha(0)
ax = fig.add_subplot(111)
for values in data:
    ax.pie(values, colors=("#e6194b", "#00d29f", '#ff9300'))
    buff = io.StringIO()
    plt.savefig(buff, format="SVG")
    buff.seek(0)
    svg = buff.read()
    svg = svg.replace("\n", "")
    plots.append(svg)
    plt.cla()
plt.clf()
plt.close()

# Add markers for each row in df2
for i, r in df2.iterrows():
    folium.Marker([r['Long'], r['Lat']],
    # Add marker with pie chart
    icon = folium.DivIcon(html=plots[i]),
    tooltip=r['Hospital'],
    popup = folium.Popup('''
                        <p style='text-align:center; 
                        font-size:15px; 
                        font-family:courier;
                        font-weight: bold'>Valores:</p>
                        
                        <p style='color:#e6194b; text-align:center'>■  {}</p>
                        <p style='color:#00d29f; text-align:center'>■  {}</p>
                        <p style='color:#ff9300; text-align:center'>■  {}</p>
                         '''.format(data[i][0], 
                                    data[i][1], 
                                    data[i][2]))
    ).add_to(consentimiento)

# Save map as 'html'
#m.save("index.html")
    
# Show map
m